In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.io import loadmat
from scipy.signal import butter, filtfilt

EEG_FOLDER = f'../datasets/EEG_data_for_Mental_Attention_State_Detection'

bands = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 14),
    'beta': (14, 32),
    'gamma': (32, 60),
}

data_root = f'{EEG_FOLDER}/EEG_Data'
channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']


In [3]:


def bp_filter(data, f_lo, f_hi, fs):
    """ Digital band pass filter (6-th order Butterworth)
    Args:
        data: numpy.array, time along axis 0
        (f_lo, f_hi): frequency band to extract [Hz]
        fs: sampling frequency [Hz]
    Returns:
        data_filt: band-pass filtered data, same shape as data """
    data_filt = np.zeros_like(data)
    f_ny = fs / 2.  # Nyquist frequency
    b_lo = f_lo / f_ny  # normalized frequency [0..1]
    b_hi = f_hi / f_ny  # normalized frequency [0..1]
    # band-pass filter parameters
    p_lp = {"N":6, "Wn":b_hi, "btype":"lowpass", "analog":False, "output":"ba"}
    p_hp = {"N":6, "Wn":b_lo, "btype":"highpass", "analog":False, "output":"ba"}
    bp_b1, bp_a1 = butter(**p_lp)
    bp_b2, bp_a2 = butter(**p_hp)
    data_filt = filtfilt(bp_b1, bp_a1, data, axis=0)
    data_filt = filtfilt(bp_b2, bp_a2, data_filt, axis=0)
    return data_filt


#  Function to read in the EEG data and extract the valid lead data, low and high pass filter and z-transform the data.
#  Returns a dataframe.
def get_EEG_data(data_root, filename):
    # Extract the data from one of these files.
    hz = fs
    #filename = 'eeg_record30.mat'
    mat = loadmat(f'{data_root}/{filename}')
    data = pd.DataFrame.from_dict(mat["o"]["data"][0,0])

    # Limit the data to the 7 valid EEG leads.
    dat = data.filter(list(range(3, 17)))
    dat.columns = list(range(1, 15))
    dat = dat.filter([1,2, 3, 4,5,6, 7, 8, 9,10,11,12,13,14,17], axis=1)
    labels = ['AF3','F7', 'F3','FC5','T7','P7','O1', 'O2','P8','T8', 'FC6','F4','F8','AF4']  # FP2 should really be AF4
    dat.columns = labels

    # Filter the data, high pass .5 Hz, low pass 62 Hz.
    lo, hi = .5, 62
    # Do the filtering.
    datf = bp_filter(dat.to_numpy(), lo, hi, hz)

    # Convert back to a dataframe.
    dat = pd.DataFrame({c: datf[:, i] for i, c in enumerate(labels)})

    # Z-transform each column
    # dat = dat.apply(zscore)

    return dat

def get_trial_data(trial):
    dat = get_EEG_data(data_root, f'eeg_record{trial}.mat')
    return dat

def plot_spectrogram(data, channel_name, fs, fmin, fmax, show=True):
    # plot the spectrogram
    plt.figure(figsize=(14, 5))
    Sxx, freqs, bins, im = plt.specgram(data, NFFT=2*fs, Fs=fs,
                                         noverlap=fs, cmap='jet')
    if show:
        plt.ylim(fmin, fmax)
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
        plt.title(f'ch={channel_name} fs={fs} band=({fmin} - {fmax})Hz')
        plt.imshow(Sxx, aspect='auto',
                   cmap='jet', origin='lower',
                   extent=[bins[0], bins[-1], freqs[0], freqs[-1]],
                   vmin=0, vmax=np.max(Sxx[(freqs >= fmin) & (freqs <= fmax)]))
        plt.colorbar()
    else:
        plt.close()
    return Sxx

In [7]:
fs = 128

class Subject:

    def __init__(self, number, slice_from, slice_to, sample_length=10, trials=None):
        self.number = number
        # records = np.arange(1, 7) + (number * 7)
        trials = trials or np.arange(3, 7) + (number * 7)
        samples = np.arange(0, slice_to - slice_from, sample_length)
        self.focus_label = Label('focus', samples, trials, slice_from, slice_to, sample_length)
        self.unfocus_label = Label('unfocus', samples, trials, slice_from + 600, slice_to + 600, sample_length)
        self.drowsed_label = Label('drowsed', samples, trials, slice_from + 1200, slice_to + 1200, sample_length)


class Label:

    def __init__(self, label, samples, trials, slice_from, slice_to, sample_length):
        self.label = label
        self.slice_to = slice_to
        self.slice_from = slice_from
        self.sample_length = sample_length

        topographic = {ch: np.zeros((len(samples) * len(trials), fs + 1, sample_length))
                       for ch in channels}
        time_signals = {ch: np.zeros((len(samples) * len(trials), fs * sample_length))
                       for ch in channels}
        for i, trial in enumerate(trials):
            trial_data = get_trial_data(trial)
            for ch in trial_data.columns:
                fft_spectogram = plot_spectrogram(trial_data[ch][slice_from * fs:slice_to * fs], channel_name=ch, fs=fs, fmin=0, fmax=63, show=False)
                for j, sample in enumerate(samples):
                    # fft_spectogram.shape[1] is 129, probably not divided by sample length.
                    actual_sample_length = fft_spectogram[:, sample:sample+sample_length].shape[1]
                    topographic[ch][i * len(samples) + j, :, :actual_sample_length] = fft_spectogram[:, sample:sample+sample_length]

                    # time domain signal
                    time_signals[ch][i * len(samples) + j, :] = trial_data[ch][(slice_from + sample) * fs:(slice_from + sample+sample_length) * fs]

        self.topographic = topographic
        self.time_signals = time_signals

slice_from = 2 * 60
slice_to = 9 * 60
subject0 = Subject(0, slice_from, slice_to)

In [18]:
def create_data_set(trials):
    dfs = []
    for trial in trials:
        trial_data = get_trial_data(trial).iloc[:30 * 60 * fs]
        trial_data['ts_sample'] = trial_data.index
        trial_data['trial'] = trial
        trial_data['subject'] = trial // 7
        trial_data['label'] = trial_data.index // (10 * 60 * fs)
        dfs.append(trial_data)
    df = pd.concat(dfs, ignore_index=True)
    return df

create_data_set([3, 4, 5, 6, 7])

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,trial,subject,label
0,-0.247007,18.670033,3.900146,0.652039,-0.246780,-0.184585,8.675864,10.274071,9.480108,0.115634,1.892281,0.043041,-0.226046,0.221301,3,0,0
1,-0.258648,11.527584,-0.650376,-1.367125,-0.254026,-1.335905,4.910812,3.070960,1.153778,0.115514,5.519147,0.044933,1.310674,-6.156922,3,0,0
2,-0.260499,14.701540,1.603110,-0.877743,-0.269944,-0.924190,11.538970,9.392245,4.239776,0.116566,9.771870,0.042280,0.779377,-5.297217,3,0,0
3,-0.271885,17.770998,5.292383,2.212911,-0.276852,0.532179,12.323887,8.215093,7.214644,0.115973,7.750340,0.043484,0.282664,-2.468666,3,0,0
4,-0.274323,16.335641,6.026544,3.234324,-0.293748,0.948941,11.740531,2.745537,3.659361,0.117091,6.888093,0.041343,-0.266346,-4.183399,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230395,0.089810,-6.359596,-5.997430,-0.145753,-2.194530,-3.102373,-2.816104,-2.122354,-7.645962,2.037864,-1.782552,0.000672,0.434419,-7.729285,7,1,2
230396,1.686302,-4.560892,-5.542580,2.413257,-1.286725,-3.032275,-2.659020,-1.447164,-3.993928,1.084075,-2.321527,0.000666,-0.523988,-8.368546,7,1,2
230397,1.107630,-3.244172,-6.675132,1.904809,-1.694754,-2.519104,-3.013036,2.431595,-1.076446,-0.012715,-2.810249,0.000667,-1.621234,-7.701372,7,1,2
230398,-0.890659,-8.627278,-7.244742,-1.692439,-1.304308,-7.556040,-10.538485,-3.552073,-11.780192,0.056634,-1.805568,0.000661,-1.548866,-6.789958,7,1,2


In [5]:


# Iterate through the subplots
def create_df_features(subject):

    labels = [subject.focus_label, subject.unfocus_label, subject.drowsed_label]

    df_mean_data = {
        f'{inst_label.label}_{ch}': []
        for inst_label in labels
        for ch in inst_label.time_signals.keys()
    }

    df_max_data = {
        f'{inst_label.label}_{ch}': []
        for inst_label in labels
        for ch in inst_label.time_signals.keys()
    }

    df_std_data = {
        f'{inst_label.label}_{ch}': []
        for inst_label in labels
        for ch in inst_label.time_signals.keys()
    }

    for j, label_inst in enumerate(labels):
        for i, (ch, sig) in enumerate(label_inst.time_signals.items()):
            for sample in sig:
                df_mean_data[f'{label_inst.label}_{ch}'].append(np.mean(sample))
                df_max_data[f'{label_inst.label}_{ch}'].append(np.max(sample))
                df_std_data[f'{label_inst.label}_{ch}'].append(np.std(sample))

    df_mean = pd.DataFrame(df_mean_data)
    df_max = pd.DataFrame(df_max_data)
    df_std = pd.DataFrame(df_std_data)

    return df_mean, df_max, df_std


def create_ch_feature(subject, label, feature):
    feature_id = {
        'mean': 0,
        'max': 1,
        'std': 2
    }[feature]
    df = create_df_features(subject)[feature_id]
    selected_columns = df.columns[df.columns.str.startswith(label)]
    df = df[selected_columns]
    ch_columns = np.array(list(df.columns.str.split('_').values))[:, 1]
    df.columns = ch_columns
    return df

def check_ch_activation(subject, label, feature, threshold):
    df = create_ch_feature(subject, label, feature)
    df = df > threshold

    row_counts = df.apply(lambda row: '-'.join(df.columns[row]), axis=1).value_counts().to_dict()
    return row_counts

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

for sample_length in [10, 5, 3, 1]:
    # subjects = [Subject(i, slice_from, slice_to, sample_length=sample_length) for i in range(5)]
    subjects = [Subject(0, slice_from, slice_to, sample_length=sample_length, trials=[3])]
    concatenated_dfs = []
    for subject in subjects:
        df1 = create_ch_feature(subject, 'focus', 'std')
        df1['label'] = 'focus'
        df2 = create_ch_feature(subject, 'unfocus', 'std')
        df2['label'] = 'unfocus'
        df3 = create_ch_feature(subject, 'drowsed', 'std')
        df3['label'] = 'drowsed'
        concatenated_dfs.append(pd.concat([df1, df2, df3], ignore_index=True))

    concatenated_df = pd.concat(concatenated_dfs, ignore_index=True)
    X = concatenated_df[channels]
    y = concatenated_df['label']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    dt_classifier = RandomForestClassifier()
    dt_classifier.fit(X_train, y_train)

    predictions = dt_classifier.predict(X_val)
    acc = np.sum(predictions == y_val)/len(y_val)
    print(f'For sample length of {sample_length}s the accuracy is {acc*100:.2f}')

For sample length of 10s the accuracy is 88.46
For sample length of 5s the accuracy is 96.08
For sample length of 3s the accuracy is 92.86
For sample length of 1s the accuracy is 89.68
